In [5]:
import os
import numpy as np
from PIL import Image
import random
import math
import pandas as pd

In [6]:
# 1. Carica l’immagine e normalizza in [0,1]
img = Image.open('cat.jpg').convert('RGB')
img_arr = np.asarray(img, dtype=np.float32) / 255.0


b = 0.17
a = -b
noisy_arr = add_uniform_noise(img_arr,a,b)
noisy_img = Image.fromarray((noisy_arr * 255).astype(np.uint8))
var = (b-a)*(b-a)/12
print("var:",var)
# 4a. Salva su disco
#noisy_img.save('noisy_image.jpg')

# 4b. Oppure mostra a video (apre il viewer predefinito)
noisy_img.show()
#Andiamo
60/255

NameError: name 'add_uniform_noise' is not defined

In [10]:
def add_salt_pepper_noise(image: np.ndarray, amount: float = 0.05, salt_vs_pepper: float = 0.5) -> np.ndarray:
    """
    :param image: Input image as float32 numpy array in [0,1], shape (H, W, 3).
    :param amount: Fraction of total pixels to corrupt.
    :param salt_vs_pepper: Fraction of corrupted pixels that are salt (vs pepper).
    :return: Noisy image as numpy array.
    """
    noisy = np.copy(image)
    ROWS, COLS, _ = image.shape
    num_pixels = ROWS * COLS
    num_salt = int(np.ceil(amount * num_pixels * salt_vs_pepper))
    num_pepper = int(np.ceil(amount * num_pixels * (1 - salt_vs_pepper)))

    # SALT
    coords_y = np.random.randint(0, ROWS, num_salt)
    coords_x = np.random.randint(0, COLS, num_salt)
    noisy[coords_y, coords_x, :] = 1.0

    # PEPPER
    coords_y = np.random.randint(0, ROWS, num_pepper)
    coords_x = np.random.randint(0, COLS, num_pepper)
    noisy[coords_y, coords_x, :] = 0.0

    return noisy



def add_gaussian_noise(image: np.ndarray, mean: float = 0.0, var: float = 0.01) -> np.ndarray:
    """
    :param image: Input image as a float32 numpy array with values in [0, 1].
    :param mean: Mean of Gaussian noise.
    :param var: Variance of Gaussian noise.
    :return: Noisy image as numpy array.
    """
    sigma = math.sqrt(var)
    gauss = np.random.normal(mean, sigma, image.shape)
    noisy = image + gauss
    return np.clip(noisy, 0, 1)



def add_uniform_noise(image: np.ndarray, low: float = -0.05, high: float = 0.05) -> np.ndarray:
    """
    :param image: Input image as a float32 numpy array with values in [0, 1].
    :param low: Lower bound of uniform noise.
    :param high: Upper bound of uniform noise.
    :return: Noisy image as numpy array.
    """
    uniform_noise = np.random.uniform(low, high, image.shape)
    noisy = image + uniform_noise
    return np.clip(noisy, 0, 1)


def add_erlang_noise(image: np.ndarray, k: int, lam: float) -> np.ndarray:
    """
    Add Erlang (Gamma) noise to an image, using shape k and rate λ.
    The Erlang PDF is: f(x) = (λ^k * x^(k-1) / (k-1)!) * e^{-λ x},  x >= 0

    :param image:     Input image as float32 numpy array in [0,1], shape (H, W, C).
    :param k:         Shape parameter (integer ≥ 1).
    :param lam:       Rate parameter λ (> 0).
    :return:          Noisy image as numpy array, clipped to [0,1].
    """
    scale = 1.0 / lam   # scale θ = 1/λ
    noise = np.random.gamma(shape=k, scale=scale, size=image.shape).astype(np.float32) # Genero rumore Gamma(k, θ)
    noise_zero_mean = noise - (k * scale) # Centro a media zero  (E[Gamma] = k*θ = k/λ)
    # add to image and clip
    noisy = image + noise_zero_mean
    return np.clip(noisy, 0.0, 1.0)

In [11]:
# 1. Carica e normalizza
img = Image.open('cat.jpg').convert('RGB')
arr = np.asarray(img, dtype=np.float32) / 255.0

noisy_arr = add_erlang_noise(arr, k=3, lam=25)
# 12,25
# 3. Riconverti e salva
noisy_img = Image.fromarray((noisy_arr * 255).astype(np.uint8))
noisy_img.show()
#noisy_img.save('path/to/noisy_erlang.jpg')

Bisogna calcolare la varianza per ogni rumore, e settare i parametri affinchè la varianza media sia 0.01.

In [12]:
def generate_dataset(input_dir: str, output_dir: str) -> None:
    """
    Genera un dataset con:
      - originali
      - salt & pepper (amount random in [0.01,0.10])
      - gaussian    (var random in [0.005,0.02], mean=0)
      - uniform     (r random in [0.10,0.20], noise in [-r,+r])
      - erlang      (k=3, λ random in [12,25])
    e salva un CSV con filepath e label.
    """
    # Range dei parametri
    sp_range    = (0.01, 0.10)
    gauss_range = (0.005, 0.02)
    uni_range   = (0.10, 0.20)
    erlang_range = (12.0, 25.0)

    label_map = {
        'original':    'Original',
        'salt_pepper': 'Salt & Pepper',
        'gaussian':    'Gaussian',
        'uniform':     'Uniform',
        'erlang':      'Erlang'
    }

    # Crea cartelle
    for folder in label_map.keys():
        os.makedirs(os.path.join(output_dir, folder), exist_ok=True)

    records = []
    for fname in os.listdir(input_dir):
        if not fname.lower().endswith(('.png','.jpg','.jpeg','.bmp','.tiff')):
            continue

        # Carica e normalizza [0,1]
        img = Image.open(os.path.join(input_dir, fname)).convert('RGB')
        arr = np.asarray(img, dtype=np.float32) / 255.0

        # --------------------------------------------------
        # Originale
        orig_path = os.path.join(output_dir, 'original', fname)
        img.save(orig_path)
        records.append({'filepath': orig_path, 'label': label_map['original']})

        # Salt & Pepper
        amount = random.uniform(*sp_range)
        sp_arr = add_salt_pepper_noise(arr, amount=amount, salt_vs_pepper=0.5)
        sp_img = Image.fromarray((sp_arr*255).astype(np.uint8))
        sp_path = os.path.join(output_dir, 'salt_pepper', fname)
        sp_img.save(sp_path)
        records.append({'filepath': sp_path, 'label': label_map['salt_pepper']})

        # Gaussian
        var = random.uniform(*gauss_range)
        g_arr = add_gaussian_noise(arr, mean=0.0, var=var)
        g_img = Image.fromarray((g_arr*255).astype(np.uint8))
        g_path = os.path.join(output_dir, 'gaussian', fname)
        g_img.save(g_path)
        records.append({'filepath': g_path, 'label': label_map['gaussian']})

        # Uniform
        r = random.uniform(*uni_range)
        u_arr = add_uniform_noise(arr, low=-r, high=+r)
        u_img = Image.fromarray((u_arr*255).astype(np.uint8))
        u_path = os.path.join(output_dir, 'uniform', fname)
        u_img.save(u_path)
        records.append({'filepath': u_path, 'label': label_map['uniform']})

        # Erlang (Gamma)
        lam = random.uniform(*erlang_range)
        e_arr = add_erlang_noise(arr, k=3, lam=lam)
        e_img = Image.fromarray((e_arr*255).astype(np.uint8))
        e_path = os.path.join(output_dir, 'erlang', fname)
        e_img.save(e_path)
        records.append({'filepath': e_path, 'label': label_map['erlang']})

    # Salva CSV delle label
    df = pd.DataFrame(records)
    csv_path = os.path.join(output_dir, 'labels.csv')
    df.to_csv(csv_path, index=False)
    print(f"Dataset and labels salvati. CSV scritto in: {csv_path}")

In [13]:
# ESEGUO
input_directory = "CocoMiniMini_100/."
output_directory = "Dataset_100x5."
generate_dataset(input_directory, output_directory)

Dataset and labels salvati. CSV scritto in: Dataset_100x5.\labels.csv
